# CS4287 Neural Computing - Assignment 2: Convolutional Neural Networks

**Team Members:** [INSERT NAMES AND ID NUMBERS]  
**Student ID 1:** [INSERT]  
**Student ID 2:** [INSERT]

**Code Execution Status:** [Comment on whether the code executes to completion without errors]

**Third Party Source:** [Provide link to any existing implementation used]


## Table of Contents
1. [Imports and Setup](#imports)
2. [Data Loading and Preprocessing](#data)
3. [Network Architecture](#architecture)
4. [Cost Function](#loss)
5. [Optimizer](#optimizer)
6. [Cross-Fold Validation](#validation)
7. [Training and Results](#results)
8. [Hyperparameter Analysis](#hyperparameters)
9. [Evaluation](#evaluation)


## 1. Imports and Setup {#imports}


In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# Deep learning frameworks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, losses, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Sklearn for evaluation and preprocessing
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


## 2. Data Loading and Preprocessing {#data}


In [ ]:
# Load Fruit Detection Dataset from Kaggle
# Dataset: lakshaytyagi01/fruit-detection
# Reorganized from YOLO format to classification format with 6 classes:
# Apple, Banana, Grape, Orange, Pineapple, Watermelon

DATASET_PATH = "data/fruits_classification"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Load training dataset - Keras automatically creates labels from folder names
train_dataset = keras.utils.image_dataset_from_directory(
    f'{DATASET_PATH}/train',
    labels='inferred',
    label_mode='categorical',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

# Load test dataset
test_dataset = keras.utils.image_dataset_from_directory(
    f'{DATASET_PATH}/test',
    labels='inferred',
    label_mode='categorical',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42
)

# Load validation dataset
val_dataset = keras.utils.image_dataset_from_directory(
    f'{DATASET_PATH}/valid',
    labels='inferred',
    label_mode='categorical',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42
)

# Get class names and number of classes
class_names = train_dataset.class_names
num_classes = len(class_names)

print(f"\nDataset loaded successfully!")
print(f"Number of classes: {num_classes}")
print(f"Fruit classes: {class_names}")
print(f"Number of training batches: {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Number of test batches: {tf.data.experimental.cardinality(test_dataset).numpy()}")
print(f"Number of validation batches: {tf.data.experimental.cardinality(val_dataset).numpy()}")

# Store image shape for later use
input_shape = (*IMAGE_SIZE, 3)
print(f"Input shape: {input_shape}")


### 2.1 Data Visualization

Visualize sample fruit images and their distribution


In [ ]:
# Visualize sample fruits from training set
plt.figure(figsize=(15, 15))

# Get first batch from training data
for images, labels in train_dataset.take(1):
    # Display 16 sample images
    for i in range(min(16, len(images))):
        plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # Display image
        # Get class name from one-hot encoded label
        label_idx = np.argmax(labels[i].numpy())
        plt.title(f"{class_names[label_idx]}", fontsize=10)
        plt.axis('off')

plt.suptitle('Sample Fruits from Training Set', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Plot class distribution
class_counts = {}
total_samples = 0

# Count samples per class
for images, labels in train_dataset:
    for label in labels:
        idx = np.argmax(label)
        class_name = class_names[idx]
        class_counts[class_name] = class_counts.get(class_name, 0) + len(images)
    total_samples += len(images)

# Visualize distribution
plt.figure(figsize=(14, 6))
bars = plt.bar(class_counts.keys(), class_counts.values(), color='steelblue', edgecolor='black')
plt.title('Distribution of Fruit Classes in Training Set', fontsize=14, fontweight='bold')
plt.xlabel('Fruit Class', fontsize=12)
plt.ylabel('Number of Images', fontsize=12)
plt.xticks(rotation=45, ha='right')

# Add count labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nClass Distribution:")
for class_name, count in sorted(class_counts.items()):
    percentage = (count / sum(class_counts.values())) * 100
            print(f"  {class_name}: {count} images ({percentage:.1f}%)")


## 3. CNN Architecture Definition {#architecture}

Define the Convolutional Neural Network architecture for fruit detection.


In [ ]:
# Build CNN Model for Fruit Detection
def build_cnn_model(input_shape, num_classes):
    """
    Build a CNN model for fruit detection and classification.
    
    This architecture uses:
    - He weight initialization (He et al., 2015) for ReLU activations
    - Batch normalization (Ioffe & Szegedy, 2015) to stabilize training
    - Max pooling for dimensionality reduction
    - Dropout (Srivastava et al., 2014) for regularization
    - Softmax activation for multi-class classification
    """
    model = models.Sequential([
        # First Convolutional Block - extracts basic visual features (edges, colors)
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape,
                     kernel_initializer='he_normal', name='conv1_1'),  # He initialization
        layers.BatchNormal要提高lization(name='bn1_1'),  # Batch norm for stability
        layers.Conv2D(32, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv1_2'),
        layers.BatchNormalization(name='bn1_2'),
        layers.MaxPooling2D((2, 2), name='pool1'),  # Spatial dimension reduction
        layers.Dropout(0.25, name='dropout1'),       # Regularization
        
        # Second Convolutional Block - extracts textures and patterns
        layers.Conv2D(64, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv2_1'),
        layers.BatchNormalization(name='bn2_1'),
        layers.Conv2D(64, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv2_2'),
        layers.BatchNormalization(name='bn2_2'),
        layers.MaxPooling2D((2, 2), name='pool2'),
        layers.Dropout(0.25, name='dropout2'),
        
        # Third Convolutional Block - fruit-specific high-level features
        layers.Conv2D(128, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv3_1'),
        layers.BatchNormalization(name='bn3_1'),
        layers.Conv2D(128, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv3_2'),
        layers.BatchNormalization(name='bn3_2'),
        layers.MaxPooling2D((2, 2), name='pool3'),
        layers.Dropout(0.25, name='dropout3'),
        
        # Fourth Convolutional Block - very high-level representations
        layers.Conv2D(256, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv4_1'),
        layers.BatchNormalization(name='bn4_1'),
        layers.Conv2D(256, (3, 3), activation='relu',
                     kernel_initializer='he_normal', name='conv4_2'),
        layers.BatchNormalization(name='bn4_2'),
        layers.MaxPooling2D((2, 2), name='pool4'),
        layers.Dropout(0.25, name='dropout4'),
        
        # Fully Connected Layers - classification
        layers.Flatten(name='flatten'),              # Flatten for FC layers
        layers.Dense(512, activation='relu',                # Hidden layer 1
                    kernel_initializer='he_normal', name='fc1'),
        layers.BatchNormalization(name='bn_fc1'),
        layers.Dropout(0.5, name='dropout_fc1'),            # Higher dropout in FC
        layers.Dense(256, activation='relu',                # Hidden layer 2
                    kernel_initializer='he_normal', name='fc2'),
        layers.BatchNormalization(name='bn_fc2'),
        layers.Dropout(0.5, name='dropout_fc2'),
        
        # Output Layer - probability distribution over fruit classes
        layers.Dense(num_classes, activation='softmax', name='output')  # Softmax for multi-class
    ])
    
    return model

# Build the model
print("Building fruit detection CNN model...")
model = build_cnn_model(input_shape, num_classes)

# Display model architecture
model.summary()

        print(f"\nModel built with {model.count_params():,} parameters")
